In [1]:
import requests
import time

In [4]:
clef = 'RGAPI-10062e3a-ad75-4038-9101-786837289f98'

In [68]:
req = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_6429580663?api_key=RGAPI-88dc05df-ad27-4106-9d6e-9c9688407ea5')

In [69]:
match_info = req.json()

In [70]:
match_info

{'status': {'message': 'Data not found - match file not found',
  'status_code': 404}}

In [27]:
['individualPosition','championId',"win"]

'SOLO'

In [5]:
Matches = []

In [3]:
def add_matchs(id_base,api_key):
    req = 'https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_{}?api_key={}'.format(id_base,api_key)
    r = requests.get(req)
    match_info = r.json()
    if match_info['info']['queueId']==420:
        Blue,Red = [],[]
        players = match_info['info']['participants']
        Winner = "Blue" if players[0]['win'] else "Red"
        for i in range(len(players)):
            if i<5:
                Blue.append(players[i]['individualPosition'])
                Blue.append(players[i]['championName'])
            else:
                Red.append(players[i]['individualPosition'])
                Red.append(players[i]['championName'])
        Matches.append([Blue,Red,Winner])

In [4]:
print(Matches)
add_matchs(6429580661,'RGAPI-88dc05df-ad27-4106-9d6e-9c9688407ea5')
print(Matches)

[]
[[['TOP', 'Riven', 'JUNGLE', 'Evelynn', 'MIDDLE', 'Zed', 'BOTTOM', 'Jhin', 'UTILITY', 'Lux'], ['TOP', 'Chogath', 'JUNGLE', 'Khazix', 'MIDDLE', 'Orianna', 'BOTTOM', 'Caitlyn', 'UTILITY', 'Heimerdinger'], 'Blue']]


In [6]:
def add_100_match(id_base,api_key):
    c = 0
    while c<10000:
        try:
            add_matchs(id_base,api_key)
        except:
            pass
        time.sleep(0.9)
        id_base+=1
        c+=1

In [7]:
add_100_match(6429580661,'RGAPI-88dc05df-ad27-4106-9d6e-9c9688407ea5')

KeyboardInterrupt: 

In [8]:
len(Matches)

540

In [9]:
Matches

[[['TOP',
   'Riven',
   'JUNGLE',
   'Evelynn',
   'MIDDLE',
   'Zed',
   'BOTTOM',
   'Jhin',
   'UTILITY',
   'Lux'],
  ['TOP',
   'Chogath',
   'JUNGLE',
   'Khazix',
   'MIDDLE',
   'Orianna',
   'BOTTOM',
   'Caitlyn',
   'UTILITY',
   'Heimerdinger'],
  'Blue'],
 [['TOP',
   'Yone',
   'JUNGLE',
   'Nocturne',
   'MIDDLE',
   'Malzahar',
   'BOTTOM',
   'Tristana',
   'UTILITY',
   'Nami'],
  ['TOP',
   'TahmKench',
   'JUNGLE',
   'Belveth',
   'MIDDLE',
   'Neeko',
   'BOTTOM',
   'Varus',
   'UTILITY',
   'Karma'],
  'Blue'],
 [['TOP',
   'Nautilus',
   'JUNGLE',
   'Kayn',
   'MIDDLE',
   'Katarina',
   'BOTTOM',
   'Jinx',
   'UTILITY',
   'Pyke'],
  ['TOP',
   'Camille',
   'JUNGLE',
   'Kindred',
   'MIDDLE',
   'Sylas',
   'BOTTOM',
   'Aphelios',
   'UTILITY',
   'Thresh'],
  'Red'],
 [['TOP',
   'Garen',
   'JUNGLE',
   'Warwick',
   'MIDDLE',
   'Nautilus',
   'BOTTOM',
   'Caitlyn',
   'UTILITY',
   'Milio'],
  ['TOP',
   'Kayle',
   'JUNGLE',
   'Amumu',
   'MIDDLE'